In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
#https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_split.py#L65

In [2]:
#Variable definitions...
n_splits = 5 
data = pd.DataFrame({'W':['EA-01','EA-02','EA-03','EA-04','EA-05'],'D':[1000,1000.5,1001,1001.5,1002],'G':[47.1,45.3,45.7,67.6,87.5]})
data

,W,D,G
0,EA-01,1000.0,47.1
1,EA-02,1000.5,45.3
2,EA-03,1001.0,45.7
3,EA-04,1001.5,67.6
4,EA-05,1002.0,87.5


In [3]:
#Rough Work...
np.argsort([1,0,2,1])

array([1, 0, 3, 2])

In [4]:
def gkf_split(n_splits,groups):
    unique_grps, groups = np.unique(groups,return_inverse=True)
    num_groups = len(unique_grps)
    cv = []
    
    if(n_splits>num_groups):
        raise ValueError("Error..")
    
    n_samples_per_group = np.bincount(groups)
    indices = np.argsort(n_samples_per_group)[::-1] #Reverse of the indices..
    print(indices)
    n_samples_per_group = n_samples_per_group[indices] #values sorted in desc..
    
    n_samples_per_fold = np.zeros(n_splits) #Create 0's array of specified length.
    group_to_fold = np.zeros(num_groups) #Similarly here..
    
    for it, weight in enumerate(n_samples_per_group):
        lightest_fold = np.argmin(n_samples_per_fold)
        n_samples_per_fold[lightest_fold]+=weight
        group_to_fold[indices[it]] = lightest_fold
    
    which_grp_belong_to_which_fold = group_to_fold[groups]
    indices_train_test_split = np.arange(len(groups))
    
    for i in range(n_splits):
        test_indices = np.where(which_grp_belong_to_which_fold==i)[0]
        test_mask = np.zeros(len(groups), dtype=np.bool)
        test_mask[test_indices] = True
        train_indices = indices_train_test_split[np.logical_not(test_mask)]
        cv.append((train_indices,test_indices))
    return(cv)

In [5]:
class cv:
    def __init__(self,n_splits):
        self.n_splits = n_splits
        if(self.n_splits<=1):
            raise ValueError("Error..")
    def split(self,groups):
        train_test_splits_well = gkf_split(self.n_splits,groups)
        return(train_test_splits_well)

In [6]:
cv_obj = cv(5)
wn = data['W']
groups = [0,1,2,3,4]
print(cv_obj.split(groups))

[4 3 2 1 0]
[(array([0, 1, 2, 3]), array([4])), (array([0, 1, 2, 4]), array([3])), (array([0, 1, 3, 4]), array([2])), (array([0, 2, 3, 4]), array([1])), (array([1, 2, 3, 4]), array([0]))]


In [7]:
group_kfold = GroupKFold(n_splits=3)
for train_index, test_index in group_kfold.split(data.iloc[:,:-1], data['G'], [0,1,2,3,4]):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [0 2 3] TEST: [1 4]
TRAIN: [1 2 4] TEST: [0 3]
TRAIN: [0 1 3 4] TEST: [2]


In [8]:
"""
Pipeline Resource Useful links (In order):
https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/pipeline.py
https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/metaestimators.py
https://scikit-learn.org/stable/modules/compose.html#pipeline
https://github.com/scikit-learn/scikit-learn/blob/b194674c4/sklearn/utils/validation.py#L164
https://scikit-learn.org/stable/modules/model_persistence.html
"""

'\nPipeline Resource Useful links (In order):\nhttps://github.com/scikit-learn/scikit-learn/blob/master/sklearn/pipeline.py\nhttps://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/metaestimators.py\nhttps://scikit-learn.org/stable/modules/compose.html#pipeline\nhttps://github.com/scikit-learn/scikit-learn/blob/b194674c4/sklearn/utils/validation.py#L164\nhttps://scikit-learn.org/stable/modules/model_persistence.html\n'

In [9]:
#Pipeline Rough..
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA()),('reduce_dim1', PCA(5)),('clf', SVC())]
pipe = Pipeline(estimators)

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import datasets
param_grid = dict(reduce_dim=['passthrough'],
                  clf=[SVC(),LogisticRegression()],
                  clf__C=[0.1, 10, 100])
grid_search = GridSearchCV(pipe, param_grid=param_grid)
iris = datasets.load_iris()
X = iris.data[:, :]  # we only take the first two features.
y = iris.target
grid_search.fit(X,y)
print("Fit Done!!")

/home/home/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/home/.local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: n_components=5 must be between 0 and min(n_samples, n_features)=4 with svd_solver='full'

In [11]:
#Here, I was trying out with pipe.fit() and predict and how it worked...It kinda amazing..

In [12]:
#Transformation Rough..
def fitt(a,b):
    print("All Right!!")
def Test(a,b,**c):
    print(c)
    fitt(a,b,**c)
Test(1,2,f=1)

{'f': 1}


TypeError: fitt() got an unexpected keyword argument 'f'

In [13]:
# Transformation Rough..
from sklearn.preprocessing import StandardScaler
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
scaler = StandardScaler()
print(scaler.fit_transform(data,f=1))

TypeError: fit() got an unexpected keyword argument 'f'

In [256]:
##Data Init..
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
iris = datasets.load_iris()
X = iris.data[:, :]  # we only take the first two features.
print(X)
y = iris.target

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [257]:
#We aren't supporting pasthroughs...
class Pipeline:
    def __init__(self,steps):
        self.steps = steps
        self.steps_dict = np.array(self.steps).T
        self.steps_dict = dict(zip(*self.steps_dict))
    def set_params(self,param_values): #param_values are list of tuples, as opposed to native key,value pairs.
        for e_p_name,value in param_values:
            print(e_p_name)
            est_name, p_name = e_p_name.split('__')
            self.steps_dict[est_name].set_params() #To be changed according to the design of our estimator..
            #To deign my version of algo, that supports list of tuples.....
    def get_params(self):
        #Print the params of Pipeline as well as that of individual est, if deep=True.., Same for all est
        #Must be implemented for all est..
        out = self.__dict__
        for e_name, est in self.steps:
            for key,val in est.get_params().items():
                out['%s__%s'%(e_name,key)] = val
        return(out)
    def fit(self,data,y):
        for e_name, est in self.steps[:-1]:
            print(e_name,est)
            est.fit(data)
            data = est.transform(data)
        self.steps[-1][1].fit(data,y)
        print("Fitting sucess...")
    def predict(self,data):
        for e_name, est in self.steps[:-1]:
            data = est.transform(data)
        return(self.steps[-1][1].predict(data))

In [258]:
class StandardScaling():
    def __init__(self):
        pass
    def fit(self,data):
        self.mean = np.mean(data,axis=0)
        self.stdev = np.std(data,axis=0)
    def transform(self,data):
        data-=self.mean
        data/=self.stdev
        return(data)
    def get_params(self):
        return(dict()) #Since, all get_params are expected to return a dictionary..
#Ignoring set_params for this particular transformer..

In [259]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
pipe1 = Pipeline(steps=[('ftr',StandardScaler()),('clf',LogisticRegression())])
#Note, that LogReg must implement set_params, get_params...
#Moreover, pipeline must support the function of underlying algo.

In [260]:
pipe.fit(np.array(X),y)
print(X)

ftr <__main__.StandardScaling object at 0x7f47466dbe10>
Fitting sucess...
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 

/home/home/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/home/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [20]:
#Resources I Used..
#https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/metaestimators.py
#https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/pipeline.py
#https://github.com/scikit-learn/scikit-learn/blob/b194674c4/sklearn/base.py#L136
#https://scikit-learn.org/stable/modules/compose.html#pipeline
#https://github.com/scikit-learn/scikit-learn/blob/b194674c4/sklearn/preprocessing/_data.py#L520